# Libraries

In [2]:
import pandas as pd # type: ignore
from imblearn.under_sampling import RandomUnderSampler # type: ignore
import matplotlib.pyplot as plt # type: ignore
import math
import seaborn as sns # type: ignore
import random
import calendar

# Dataset Preparation

## MMSA

In [5]:
# Load the dataset
mmsa = pd.read_csv('MMSA_2021.csv')

In [23]:
# The MMSA categorical columns are described in the MMSA 2021 Codebook
mmsa_cat_columns = [
    # Sec 1 Health Status
    '_RFHLTH',

    # Sec 2 Healthy Days
    '_PHYS14D',
    '_MENT14D',

    # Sec 3 Health Care Access
    '_HLTHPLN',
    '_HCVU652',

    # Sec 4 Exercise
    '_TOTINDA',

    # Sec 5 Hypertension Awareness
    '_RFHYPE6',

    # Sec 6 Cholesterol Awareness
    '_CHOLCH3',
    '_RFCHOL3',

    # Sec 7 Chronic Health Conditions
    '_MICHD',

    # Sec 8 Arthritis
    

    # Sec 11 Tobacco Use
    '_SMOKER3',

    # Sec 12 Alcohol Consumption
    '_RFBING5',
    '_RFDRHV7',

    # Sec 14 HIV/AIDS
    '_AIDTST4',

    # Sec 15 Fruits & Vegetables
    '_FRTLT1A',
    '_VEGLT1A',
    
    # Sec 9 Demographics
    '_INCOMG1',
    '_EDUCAG',
    '_RFBMI5',
    '_BMI5CAT',
    '_AGE_G',
    '_SEX',
    '_RACE'
]

In [6]:
# The categorical columns below will be used for the join algorithms
mmsa_sel_columns = [
    # Sec 1 Health Status
    '_RFHLTH',

    # Sec 3 Health Care Access
    '_HLTHPLN',
    '_HCVU652',

    # Sec 5 Hypertension Awareness
    '_RFHYPE6',

    # Sec 6 Cholesterol Awareness
    '_CHOLCH3',
    '_RFCHOL3',

    # Sec 7 Chronic Health Conditions
    '_MICHD',

    # Sec 9 Demographics
    '_INCOMG1',
    '_AGE_G',
]

In [7]:
mmsa.shape

(227792, 152)

In [8]:
mmsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227792 entries, 0 to 227791
Columns: 152 entries, DISPCODE to MMSANAME
dtypes: float64(151), object(1)
memory usage: 264.2+ MB


In [9]:
# I select a subset of the categorical columns
mmsa = mmsa[mmsa_sel_columns]

In [10]:
mmsa.shape

(227792, 9)

In [11]:
mmsa.head()

,_RFHLTH,_HLTHPLN,_HCVU652,_RFHYPE6,_CHOLCH3,_RFCHOL3,_MICHD,_INCOMG1,_AGE_G
0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,5.0,3.0
1,1.0,1.0,1.0,1.0,1.0,1.0,2.0,6.0,4.0
2,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0
3,1.0,1.0,1.0,1.0,2.0,1.0,2.0,5.0,2.0
4,1.0,9.0,9.0,1.0,1.0,1.0,2.0,6.0,5.0


In [12]:
# Drop the rows with null values
mmsa.dropna(inplace=True)

In [13]:
mmsa.shape

(196015, 9)

In [14]:
# All the columns have float values to represent their categories. Convert them to integer
for column in mmsa_sel_columns:
    mmsa[column] = mmsa[column].astype('int64')

In [15]:
mmsa.head()

,_RFHLTH,_HLTHPLN,_HCVU652,_RFHYPE6,_CHOLCH3,_RFCHOL3,_MICHD,_INCOMG1,_AGE_G
0,1,1,1,1,1,1,2,5,3
1,1,1,1,1,1,1,2,6,4
2,1,1,1,1,1,1,2,3,3
3,1,1,1,1,2,1,2,5,2
4,1,9,9,1,1,1,2,6,5


In [16]:
mmsa['_MICHD'].value_counts()

_MICHD
2    180421
1     15594
Name: count, dtype: int64

Undersample the majority class to reduce the number of entries.

In [17]:
# Define the undersampler
undersampler = RandomUnderSampler(random_state=42)

# Separate features (X) and target variable (y)
X = mmsa.drop(columns=['_MICHD'])
Y = mmsa['_MICHD']

# Perform undersampling
X_resampled, Y_resampled = undersampler.fit_resample(X, Y)

# Construct the balanced DataFrame
dataframe_sample = pd.concat([X_resampled, Y_resampled], axis=1)

In [20]:
michd_1 = dataframe_sample[dataframe_sample['_MICHD'] == 1].sample(n=5000, random_state=42)
michd_2 = dataframe_sample[dataframe_sample['_MICHD'] == 2].sample(n=5000, random_state=42)

In [21]:
sample_reduced = pd.concat([michd_1, michd_2])

In [22]:
sample_reduced.head()

,_RFHLTH,_HLTHPLN,_HCVU652,_RFHYPE6,_CHOLCH3,_RFCHOL3,_INCOMG1,_AGE_G,_MICHD
150528,1,1,9,2,1,2,6,6,1
148377,2,1,9,1,1,1,2,6,1
123381,1,1,1,1,1,2,6,3,1
124702,1,1,9,2,1,2,5,6,1
179482,1,1,9,1,1,2,6,6,1


In [23]:
sample_reduced.shape[0]

10000

In [24]:
sample_reduced.to_excel("MMSA_2021_Undersampled_VerB.xlsx", index=False)

In [18]:
dataframe_sample.shape

(31188, 9)

In [19]:
dataframe_sample['_MICHD'].value_counts()

_MICHD
1    15594
2    15594
Name: count, dtype: int64

In [33]:
dataframe_sample.to_excel("MMSA_2021_Undersampled.xlsx", index=False)

## LLCP

In [25]:
# Load the LLCP dataset
llcp = pd.read_csv("LLCP_2021.csv")

In [26]:
# The last three columns are unique to the LLCP dataset as described in the LLCP 2021 Codebook
llcp_sel_columns = [
    '_RFHLTH',
    '_HLTHPLN',
    '_HCVU652',
    '_RFHYPE6',
    '_CHOLCH3',
    '_RFCHOL3',
    '_MICHD',
    '_INCOMG1',
    '_AGE_G',
    'MARITAL',
    'RENTHOM1',
    'EMPLOY1'

]

In [27]:
llcp = llcp[llcp_sel_columns]

In [28]:
llcp.shape

(438693, 12)

In [29]:
llcp.dropna(inplace=True)

In [30]:
llcp.shape

(370960, 12)

In [31]:
for column in llcp_sel_columns:
    llcp[column] = llcp[column].astype('int64')

In [32]:
llcp.head()

,_RFHLTH,_HLTHPLN,_HCVU652,_RFHYPE6,_CHOLCH3,_RFCHOL3,_MICHD,_INCOMG1,_AGE_G,MARITAL,RENTHOM1,EMPLOY1
0,2,1,9,1,1,2,2,3,6,1,1,7
1,1,1,9,2,1,2,1,9,6,9,1,8
2,1,1,9,2,1,1,1,2,6,3,1,7
3,1,1,1,2,1,2,2,5,5,1,1,7
4,2,1,9,1,1,2,1,2,6,1,1,8


In [33]:
llcp['_MICHD'].value_counts()

_MICHD
2    338022
1     32938
Name: count, dtype: int64

In [34]:
# Define the undersampler
undersampler = RandomUnderSampler(random_state=42)

# Separate features (X) and target variable (y)
X = llcp.drop(columns=['_MICHD'])
Y = llcp['_MICHD']

# Perform undersampling
X_resampled, Y_resampled = undersampler.fit_resample(X, Y)

# Construct the balanced DataFrame
dataframe_sample = pd.concat([X_resampled, Y_resampled], axis=1)

In [36]:
michd_1 = dataframe_sample[dataframe_sample['_MICHD'] == 1].sample(n=5000, random_state=42)
michd_2 = dataframe_sample[dataframe_sample['_MICHD'] == 2].sample(n=5000, random_state=42)

In [37]:
sample_reduced = pd.concat([michd_1, michd_2])

In [38]:
sample_reduced.to_excel("LLCP_2021_Undersampled_VerB.xlsx", index=False)

In [35]:
dataframe_sample.shape

(65876, 12)

In [44]:
dataframe_sample.to_excel("LLCP_2021_Undersampled.xlsx", index=False)

# Data Preprocessing

In [39]:
mmsa = pd.read_excel("MMSA_2021_Undersampled_VerB.xlsx")
llcp = pd.read_excel("LLCP_2021_Undersampled_VerB.xlsx")

In [40]:
llcp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   _RFHLTH   10000 non-null  int64
 1   _HLTHPLN  10000 non-null  int64
 2   _HCVU652  10000 non-null  int64
 3   _RFHYPE6  10000 non-null  int64
 4   _CHOLCH3  10000 non-null  int64
 5   _RFCHOL3  10000 non-null  int64
 6   _INCOMG1  10000 non-null  int64
 7   _AGE_G    10000 non-null  int64
 8   MARITAL   10000 non-null  int64
 9   RENTHOM1  10000 non-null  int64
 10  EMPLOY1   10000 non-null  int64
 11  _MICHD    10000 non-null  int64
dtypes: int64(12)
memory usage: 937.6 KB


In [41]:
mmsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   _RFHLTH   10000 non-null  int64
 1   _HLTHPLN  10000 non-null  int64
 2   _HCVU652  10000 non-null  int64
 3   _RFHYPE6  10000 non-null  int64
 4   _CHOLCH3  10000 non-null  int64
 5   _RFCHOL3  10000 non-null  int64
 6   _INCOMG1  10000 non-null  int64
 7   _AGE_G    10000 non-null  int64
 8   _MICHD    10000 non-null  int64
dtypes: int64(9)
memory usage: 703.2 KB


In [42]:
mmsa = mmsa.sample(frac=1, random_state=42)
llcp = llcp.sample(frac=1, random_state=42)

In [43]:
llcp.isnull().sum()

_RFHLTH     0
_HLTHPLN    0
_HCVU652    0
_RFHYPE6    0
_CHOLCH3    0
_RFCHOL3    0
_INCOMG1    0
_AGE_G      0
MARITAL     0
RENTHOM1    0
EMPLOY1     0
_MICHD      0
dtype: int64

In [44]:
mmsa.isnull().sum()

_RFHLTH     0
_HLTHPLN    0
_HCVU652    0
_RFHYPE6    0
_CHOLCH3    0
_RFCHOL3    0
_INCOMG1    0
_AGE_G      0
_MICHD      0
dtype: int64

Frequency distribution for MMSA

In [ ]:
# Split variables into groups of 3
variable_groups = [mmsa.columns[i:i+3] for i in range(0, len(mmsa.columns), 3)]

# Plot each group of variables
for group in variable_groups:
    plt.figure(figsize=(12, 6))
    for idx, variable in enumerate(group):
        plt.subplot(math.ceil(len(group)/2), 2, idx+1)
        sns.countplot(data=mmsa, x=variable, hue=variable, palette='Set3', legend=False)
        plt.title(f'Frequency of Categories in {variable}')
        plt.xlabel('Categories')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Frequency distribution for LLCP

In [ ]:
# Split variables into groups of 3
variable_groups = [llcp.columns[i:i+3] for i in range(0, len(llcp.columns), 3)]

# Plot each group of variables
for group in variable_groups:
    plt.figure(figsize=(12, 6))
    for idx, variable in enumerate(group):
        plt.subplot(math.ceil(len(group)/2), 2, idx+1)
        sns.countplot(data=llcp, x=variable, hue=variable, palette='Set3', legend=False)
        plt.title(f'Frequency of Categories in {variable}')
        plt.xlabel('Categories')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Add a timestamp to each entry of both datasets (MMSA and LLCP).

In [45]:
mmsa.shape[0]

10000

In [46]:
llcp.shape[0]

10000

In [47]:
mmsa.shape[0]/12


833.3333333333334

In [48]:
llcp.shape[0]/12

833.3333333333334

In [49]:
def generate_random_dates(month, year, num_dates):
    # Determine the number of days in the given month and year
    num_days = calendar.monthrange(year, month)[1]
    
    # Generate random dates within the given month and year
    random_dates = []
    for _ in range(num_dates):
        day = random.randint(1, num_days)
        date_str = f"{year}-{month:02d}-{day:02d}"
        random_dates.append(date_str)
    
    return random_dates

Generate random dates for each month of 2021 for the MMSA dataset. 

In [50]:
random_dates = []
for month in range(1, 13):
    random_dates.append(generate_random_dates(month, 2021, mmsa.shape[0]//12))

mmsa_timestamps = []
for month_dates in random_dates:
    mmsa_timestamps.extend(month_dates)

for date in mmsa_timestamps:
    print(date)

2021-01-18
2021-01-20
2021-01-17
2021-01-08
2021-01-19
2021-01-18
2021-01-19
2021-01-08
2021-01-01
2021-01-21
2021-01-14
2021-01-05
2021-01-06
2021-01-02
2021-01-22
2021-01-17
2021-01-31
2021-01-17
2021-01-31
2021-01-15
2021-01-10
2021-01-04
2021-01-13
2021-01-12
2021-01-10
2021-01-21
2021-01-01
2021-01-20
2021-01-26
2021-01-22
2021-01-17
2021-01-12
2021-01-04
2021-01-05
2021-01-23
2021-01-30
2021-01-23
2021-01-05
2021-01-17
2021-01-04
2021-01-01
2021-01-10
2021-01-01
2021-01-11
2021-01-02
2021-01-18
2021-01-07
2021-01-22
2021-01-09
2021-01-17
2021-01-20
2021-01-24
2021-01-13
2021-01-22
2021-01-12
2021-01-05
2021-01-03
2021-01-01
2021-01-17
2021-01-05
2021-01-02
2021-01-27
2021-01-01
2021-01-02
2021-01-19
2021-01-17
2021-01-08
2021-01-09
2021-01-20
2021-01-03
2021-01-24
2021-01-10
2021-01-21
2021-01-12
2021-01-27
2021-01-24
2021-01-24
2021-01-22
2021-01-05
2021-01-19
2021-01-25
2021-01-23
2021-01-11
2021-01-27
2021-01-06
2021-01-16
2021-01-17
2021-01-23
2021-01-17
2021-01-16
2021-01-07

In [54]:
len(mmsa_timestamps)

10000

In [52]:
mmsa.shape[0] - len(mmsa_timestamps)

4

In [53]:
mmsa_timestamps.extend(generate_random_dates(12, 2021, mmsa.shape[0] - len(mmsa_timestamps)))

In [55]:
# Convert the list of dates to a pandas Series and then to datetime
mmsa_dates = pd.to_datetime(pd.Series(mmsa_timestamps))

# Extract the month from each date
mmsa_months = mmsa_dates.dt.month

# Check if all months from 1 to 12 are present
all_months_present = set(range(1, 13)).issubset(set(mmsa_months))

if all_months_present:
    print("All months are present.")
else:
    print("Not all months are present.")

All months are present.


Generate random dates for each month of 2021 for the LLCP dataset. 

In [56]:
random_dates = []
for month in range(1, 13):
    random_dates.append(generate_random_dates(month, 2021, llcp.shape[0]//12))

llcp_timestamps = []
for month_dates in random_dates:
    llcp_timestamps.extend(month_dates)

for date in llcp_timestamps:
    print(date)

2021-01-30
2021-01-22
2021-01-28
2021-01-20
2021-01-31
2021-01-01
2021-01-02
2021-01-09
2021-01-12
2021-01-20
2021-01-01
2021-01-07
2021-01-24
2021-01-25
2021-01-04
2021-01-15
2021-01-20
2021-01-31
2021-01-27
2021-01-20
2021-01-07
2021-01-09
2021-01-15
2021-01-04
2021-01-27
2021-01-11
2021-01-13
2021-01-21
2021-01-12
2021-01-09
2021-01-26
2021-01-20
2021-01-02
2021-01-06
2021-01-02
2021-01-23
2021-01-26
2021-01-30
2021-01-15
2021-01-18
2021-01-02
2021-01-11
2021-01-06
2021-01-26
2021-01-27
2021-01-19
2021-01-03
2021-01-06
2021-01-24
2021-01-05
2021-01-29
2021-01-20
2021-01-28
2021-01-09
2021-01-14
2021-01-11
2021-01-04
2021-01-30
2021-01-07
2021-01-17
2021-01-26
2021-01-07
2021-01-23
2021-01-15
2021-01-02
2021-01-09
2021-01-24
2021-01-03
2021-01-10
2021-01-03
2021-01-29
2021-01-03
2021-01-12
2021-01-24
2021-01-31
2021-01-19
2021-01-27
2021-01-24
2021-01-08
2021-01-10
2021-01-14
2021-01-06
2021-01-30
2021-01-10
2021-01-13
2021-01-17
2021-01-28
2021-01-19
2021-01-25
2021-01-28
2021-01-06

In [57]:
len(llcp_timestamps)

9996

In [58]:
llcp.shape[0] - len(llcp_timestamps)

4

In [59]:
llcp_timestamps.extend(generate_random_dates(12, 2021, llcp.shape[0] - len(llcp_timestamps)))

In [60]:
# Convert the list of dates to a pandas Series and then to datetime
llcp_dates = pd.to_datetime(pd.Series(llcp_timestamps))

# Extract the month from each date
llcp_months = llcp_dates.dt.month

# Check if all months from 1 to 12 are present
all_months_present = set(range(1, 13)).issubset(set(llcp_months))

if all_months_present:
    print("All months are present.")
else:
    print("Not all months are present.")

All months are present.


In [61]:
mmsa['Timestamp'] = mmsa_timestamps
llcp['Timestamp'] = llcp_timestamps

In [62]:
mmsa.head()

,_RFHLTH,_HLTHPLN,_HCVU652,_RFHYPE6,_CHOLCH3,_RFCHOL3,_INCOMG1,_AGE_G,_MICHD,Timestamp
6252,1,1,1,2,1,2,3,4,2,2021-01-18
4684,2,1,9,1,1,1,3,6,1,2021-01-20
1731,1,1,9,2,1,1,3,6,1,2021-01-17
4742,1,1,1,2,1,2,1,5,1,2021-01-08
4521,2,9,9,2,1,9,9,5,1,2021-01-19


In [63]:
llcp.head()

,_RFHLTH,_HLTHPLN,_HCVU652,_RFHYPE6,_CHOLCH3,_RFCHOL3,_INCOMG1,_AGE_G,MARITAL,RENTHOM1,EMPLOY1,_MICHD,Timestamp
6252,1,1,1,1,1,1,5,5,1,1,1,2,2021-01-30
4684,2,1,1,1,1,2,1,4,1,2,8,1,2021-01-22
1731,1,1,9,1,1,1,4,6,1,1,7,1,2021-01-28
4742,1,1,9,2,1,2,5,6,1,1,7,1,2021-01-20
4521,1,1,9,2,1,2,3,6,1,1,7,1,2021-01-31


In [64]:
llcp_cp = llcp.copy()
mmsa_cp = mmsa.copy()

In [65]:
llcp_cp['Timestamp'] = pd.to_datetime(llcp_cp['Timestamp'])
mmsa_cp['Timestamp'] = pd.to_datetime(mmsa_cp['Timestamp'])

In [66]:
llcp_cp['Month'] = llcp_cp['Timestamp'].dt.month
mmsa_cp['Month'] = mmsa_cp['Timestamp'].dt.month

In [67]:
filtered_llcp = llcp_cp[(llcp_cp['_MICHD'] == 1) & (llcp_cp['Month'] == 11)]
filtered_mmsa = mmsa_cp[(mmsa_cp['_MICHD'] == 1) & (mmsa_cp['Month'] == 11)]

In [68]:
filtered_llcp.head()

,_RFHLTH,_HLTHPLN,_HCVU652,_RFHYPE6,_CHOLCH3,_RFCHOL3,_INCOMG1,_AGE_G,MARITAL,RENTHOM1,EMPLOY1,_MICHD,Timestamp,Month
1000,2,9,9,2,1,2,9,5,1,1,1,1,2021-11-18,11
3553,1,1,9,1,1,2,5,6,1,1,7,1,2021-11-13,11
4114,1,1,9,2,1,2,6,6,1,1,2,1,2021-11-14,11
1667,1,1,9,2,1,1,9,6,1,1,7,1,2021-11-24,11
3173,2,1,9,2,1,2,2,6,1,1,7,1,2021-11-13,11


In [69]:
filtered_mmsa.head()

,_RFHLTH,_HLTHPLN,_HCVU652,_RFHYPE6,_CHOLCH3,_RFCHOL3,_INCOMG1,_AGE_G,_MICHD,Timestamp,Month
1000,1,1,9,2,1,2,9,6,1,2021-11-19,11
3553,2,1,1,2,1,2,2,4,1,2021-11-04,11
4114,2,1,9,2,1,2,3,6,1,2021-11-20,11
1667,1,1,1,2,1,1,5,5,1,2021-11-11,11
3173,1,1,1,2,1,2,5,5,1,2021-11-09,11


In [70]:
# Group by the 'Month' column
grouped_llcp = llcp_cp.groupby('Month')
grouped_mmsa = mmsa_cp.groupby('Month')

In [71]:
for month, group in grouped_llcp:
    print(f"Month: {month}")
    print(f"Number of rows: {len(group)}")
    print("Values for _MICHD:")
    print(group['_MICHD'].value_counts())
    print("\n")

Month: 1
Number of rows: 833
Values for _MICHD:
_MICHD
2    426
1    407
Name: count, dtype: int64


Month: 2
Number of rows: 833
Values for _MICHD:
_MICHD
1    439
2    394
Name: count, dtype: int64


Month: 3
Number of rows: 833
Values for _MICHD:
_MICHD
1    425
2    408
Name: count, dtype: int64


Month: 4
Number of rows: 833
Values for _MICHD:
_MICHD
1    420
2    413
Name: count, dtype: int64


Month: 5
Number of rows: 833
Values for _MICHD:
_MICHD
1    460
2    373
Name: count, dtype: int64


Month: 6
Number of rows: 833
Values for _MICHD:
_MICHD
2    422
1    411
Name: count, dtype: int64


Month: 7
Number of rows: 833
Values for _MICHD:
_MICHD
2    428
1    405
Name: count, dtype: int64


Month: 8
Number of rows: 833
Values for _MICHD:
_MICHD
1    422
2    411
Name: count, dtype: int64


Month: 9
Number of rows: 833
Values for _MICHD:
_MICHD
2    443
1    390
Name: count, dtype: int64


Month: 10
Number of rows: 833
Values for _MICHD:
_MICHD
2    449
1    384
Name: count, dtyp

In [72]:
for month, group in grouped_mmsa:
    print(f"Month: {month}")
    print(f"Number of rows: {len(group)}")
    print("Values for _MICHD:")
    print(group['_MICHD'].value_counts())
    print("\n")

Month: 1
Number of rows: 833
Values for _MICHD:
_MICHD
2    426
1    407
Name: count, dtype: int64


Month: 2
Number of rows: 833
Values for _MICHD:
_MICHD
1    439
2    394
Name: count, dtype: int64


Month: 3
Number of rows: 833
Values for _MICHD:
_MICHD
1    425
2    408
Name: count, dtype: int64


Month: 4
Number of rows: 833
Values for _MICHD:
_MICHD
1    420
2    413
Name: count, dtype: int64


Month: 5
Number of rows: 833
Values for _MICHD:
_MICHD
1    460
2    373
Name: count, dtype: int64


Month: 6
Number of rows: 833
Values for _MICHD:
_MICHD
2    422
1    411
Name: count, dtype: int64


Month: 7
Number of rows: 833
Values for _MICHD:
_MICHD
2    428
1    405
Name: count, dtype: int64


Month: 8
Number of rows: 833
Values for _MICHD:
_MICHD
1    422
2    411
Name: count, dtype: int64


Month: 9
Number of rows: 833
Values for _MICHD:
_MICHD
2    443
1    390
Name: count, dtype: int64


Month: 10
Number of rows: 833
Values for _MICHD:
_MICHD
2    449
1    384
Name: count, dtyp

In [73]:
llcp.to_csv("LLCP_2021_Undersampled_VerB.csv", index=False)
mmsa.to_csv("MMSA_2021_Undersampled_VerB.csv", index=False)

# Perform Join

In [5]:
mmsa = pd.read_csv("MMSA_2021_Undersampled_VerB.csv")
llcp = pd.read_csv("LLCP_2021_Undersampled_VerB.csv")

In [6]:
mmsa.head()

,_RFHLTH,_HLTHPLN,_HCVU652,_RFHYPE6,_CHOLCH3,_RFCHOL3,_INCOMG1,_AGE_G,_MICHD,Timestamp
0,2,2,2,2,2,2,1,1,1,2021-01-18
1,1,1,9,1,1,2,6,6,2,2021-01-24
2,1,1,1,2,1,2,6,5,2,2021-01-29
3,1,1,1,2,1,1,9,3,2,2021-01-28
4,1,1,1,2,1,2,9,5,1,2021-01-29


In [7]:
llcp.head()

,_RFHLTH,_HLTHPLN,_HCVU652,_RFHYPE6,_CHOLCH3,_RFCHOL3,_INCOMG1,_AGE_G,MARITAL,RENTHOM1,EMPLOY1,_MICHD,Timestamp
0,1,1,1,1,1,2,5,5,1,1,7,1,2021-01-17
1,1,1,1,2,1,2,5,5,1,1,1,2,2021-01-01
2,1,1,9,1,1,1,9,6,1,1,5,2,2021-01-19
3,2,1,1,1,1,2,9,5,2,2,7,1,2021-01-18
4,2,1,1,2,1,1,5,4,6,2,8,1,2021-01-08


In [8]:
# Convert Timestamp to datetime
mmsa['Timestamp'] = pd.to_datetime(mmsa['Timestamp'])
llcp['Timestamp'] = pd.to_datetime(llcp['Timestamp'])

# Extract the month from Timestamp
mmsa['Month'] = mmsa['Timestamp'].dt.month
llcp['Month'] = llcp['Timestamp'].dt.month


In [ ]:
merged_mmsa_llcp = pd.merge(mmsa, llcp, on=['_MICHD', 'Month'])

In [ ]:
print(f"Number of matching rows: {merged_mmsa_llcp.shape[0]}")